In [1]:
import mlflow
import pandas as pd
from mlflow import MlflowClient

In [2]:
REMOTE_TRACKING_URI = "http://ec2-44-217-145-52.compute-1.amazonaws.com:5000"
experiment_name = "base_pattern"
mlflow.set_tracking_uri(REMOTE_TRACKING_URI)
client = MlflowClient(REMOTE_TRACKING_URI)

experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_experiment(experiment_id=experiment.experiment_id)

<Experiment: artifact_location='s3://kaggle-nakayama/mlflow/6', creation_time=1730853199485, experiment_id='6', last_update_time=1730853199485, lifecycle_stage='active', name='base_pattern', tags={}>

In [ ]:
runs = mlflow.search_runs(experiment_ids=experiment.experiment_id)
flg = runs["tags.mlflow.runName"].str.contains("lenta")
run_df = runs.loc[flg]

In [7]:
with mlflow.start_run(run_name="lenta", tags={"dataset": "lenta"}) as parent_run:
    pass

2024/11/17 05:52:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run lenta at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/6/runs/0add35cfa1224f2995bffa12ae61e17b.
2024/11/17 05:52:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/6.


In [13]:
for run_id in run_df.run_id:
    run = client.set_tag(run_id, "mlflow.parentRunId", parent_run.info.run_id)

In [25]:
import numpy as np

import numpy.typing as npt
def shape_data(
    pred: npt.NDArray[np.float_],
    y: npt.NDArray[np.float_ | np.int_],
    w: npt.NDArray[np.float_ | np.int_],
) -> pd.DataFrame:
    return (
        pd.DataFrame({"score": pred, "group": w, "conversion": y})
        .sort_values(by="score", ascending=False)
        .astype(
            {
                "score": float,
                "group": int,
                "conversion": int,
            }
        )
    )

In [42]:
pred_df = pd.read_csv("/workspace/pred.csv", index_col=0)
base_df = pd.read_csv("/workspace/base.csv", index_col=0)
_output_df = pd.merge(base_df, pred_df, left_index=True, right_index=True)

In [43]:
pred_df.set_index("index")

,pred
index,
12657168,0.000166
9588964,0.000166
9375441,0.000436
12712608,0.000166
640023,0.000166
...,...
9973311,0.004718
10916927,0.000138
2583252,0.000138


In [39]:
base_df

,y,w
6540047,0,0
12657168,0,1
9588964,0,1
4905360,0,1
7937047,0,1
...,...,...
5546691,0,1
8198879,0,1
9484578,0,1
1720671,0,1


In [32]:
pred = pd.read_csv("/workspace/pred.csv", index_col="index")
base = pd.read_csv("/workspace/base.csv", index_col=0)
df = pd.merge(base, pred, left_index=True, right_index=True)

In [33]:
df

,y,w,Unnamed: 0,pred
6540047,0,0,0,0.000177
12657168,0,1,0,0.000166
9588964,0,1,1,0.000166
4905360,0,1,0,0.000138
7937047,0,1,1,0.000089
...,...,...,...,...
5546691,0,1,279588,0.000166
8198879,0,1,279589,0.037679
9484578,0,1,279590,0.000166
1720671,0,1,279591,0.000110
